In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)
    
class Subject_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Subject_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 4)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Activity_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 3.284945011138916, Final Batch Loss: 1.0997529029846191
Epoch 2, Loss: 3.2563018798828125, Final Batch Loss: 1.0743340253829956
Epoch 3, Loss: 3.2211934328079224, Final Batch Loss: 1.0398589372634888
Epoch 4, Loss: 3.301901936531067, Final Batch Loss: 1.1309809684753418
Epoch 5, Loss: 3.266425848007202, Final Batch Loss: 1.0965145826339722
Epoch 6, Loss: 3.2476807832717896, Final Batch Loss: 1.0807243585586548
Epoch 7, Loss: 3.259092926979065, Final Batch Loss: 1.0939464569091797
Epoch 8, Loss: 3.204698920249939, Final Batch Loss: 1.0402328968048096
Epoch 9, Loss: 3.255707621574402, Final Batch Loss: 1.1032254695892334
Epoch 10, Loss: 3.250483512878418, Final Batch Loss: 1.103775978088379
Epoch 11, Loss: 3.2693684101104736, Final Batch Loss: 1.1295647621154785
Epoch 12, Loss: 3.1832327842712402, Final Batch Loss: 1.0457051992416382
Epoch 13, Loss: 3.135536551475525, Final Batch Loss: 1.0017009973526
Epoch 14, Loss: 3.2310179471969604, Final Batch Loss: 1.1151323318481445

Epoch 119, Loss: 0.279108140617609, Final Batch Loss: 0.15210559964179993
Epoch 120, Loss: 0.15095418505370617, Final Batch Loss: 0.02341247908771038
Epoch 121, Loss: 0.1694929599761963, Final Batch Loss: 0.03934955596923828
Epoch 122, Loss: 0.11873220186680555, Final Batch Loss: 0.00279886182397604
Epoch 123, Loss: 0.16262786649167538, Final Batch Loss: 0.024239489808678627
Epoch 124, Loss: 0.4212644398212433, Final Batch Loss: 0.3312135338783264
Epoch 125, Loss: 0.10758883506059647, Final Batch Loss: 0.01974828913807869
Epoch 126, Loss: 0.16237087175250053, Final Batch Loss: 0.05247543752193451
Epoch 127, Loss: 0.15345566347241402, Final Batch Loss: 0.03687919303774834
Epoch 128, Loss: 0.13703599572181702, Final Batch Loss: 0.012004919350147247
Epoch 129, Loss: 0.08323369920253754, Final Batch Loss: 0.00395621731877327
Epoch 130, Loss: 0.19369086250662804, Final Batch Loss: 0.08472877740859985
Epoch 131, Loss: 0.11419832427054644, Final Batch Loss: 0.014242415316402912
Epoch 132, Los

Epoch 241, Loss: 0.0790881747379899, Final Batch Loss: 0.012225276790559292
Epoch 242, Loss: 0.0748576195910573, Final Batch Loss: 0.008097534067928791
Epoch 243, Loss: 0.08836706727743149, Final Batch Loss: 0.004011821001768112
Epoch 244, Loss: 0.06824106629937887, Final Batch Loss: 0.00790081825107336
Epoch 245, Loss: 0.07597961788997054, Final Batch Loss: 0.004765355493873358
Epoch 246, Loss: 0.1055857203900814, Final Batch Loss: 0.03128497675061226
Epoch 247, Loss: 0.09995740279555321, Final Batch Loss: 0.008641652762889862
Epoch 248, Loss: 0.1061165239661932, Final Batch Loss: 0.01885463111102581
Epoch 249, Loss: 0.260799502953887, Final Batch Loss: 0.1780596375465393
Epoch 250, Loss: 0.17401877604424953, Final Batch Loss: 0.10471665114164352
Epoch 251, Loss: 0.09763850551098585, Final Batch Loss: 0.011844991706311703
Epoch 252, Loss: 0.09109336696565151, Final Batch Loss: 0.005849326029419899
Epoch 253, Loss: 0.12327567115426064, Final Batch Loss: 0.01954903081059456
Epoch 254, L

Epoch 349, Loss: 0.15464909747242928, Final Batch Loss: 0.06077319011092186
Epoch 350, Loss: 0.070032455958426, Final Batch Loss: 0.010233053006231785
Epoch 351, Loss: 0.21602992713451385, Final Batch Loss: 0.12887172400951385
Epoch 352, Loss: 0.045470156939700246, Final Batch Loss: 0.00034423195756971836
Epoch 353, Loss: 0.0676199090667069, Final Batch Loss: 0.00346797751262784
Epoch 354, Loss: 0.06747156847268343, Final Batch Loss: 0.011039084754884243
Epoch 355, Loss: 0.0655586221255362, Final Batch Loss: 0.003918650094419718
Epoch 356, Loss: 0.07481841964181513, Final Batch Loss: 0.001147394417785108
Epoch 357, Loss: 0.061403486877679825, Final Batch Loss: 0.011853275820612907
Epoch 358, Loss: 0.07663679198594764, Final Batch Loss: 0.0007195238140411675
Epoch 359, Loss: 0.07465857872739434, Final Batch Loss: 0.005969932768493891
Epoch 360, Loss: 0.07027029432356358, Final Batch Loss: 0.008077003061771393
Epoch 361, Loss: 0.06479293829761446, Final Batch Loss: 0.0011556174140423536


Epoch 457, Loss: 0.07140134274959564, Final Batch Loss: 0.026723256334662437
Epoch 458, Loss: 0.06728404806926847, Final Batch Loss: 0.007305872160941362
Epoch 459, Loss: 0.04692669352516532, Final Batch Loss: 0.005696484353393316
Epoch 460, Loss: 0.05920103332027793, Final Batch Loss: 0.0044790818355977535
Epoch 461, Loss: 0.06337394565343857, Final Batch Loss: 0.018218928948044777
Epoch 462, Loss: 0.07292184699326754, Final Batch Loss: 0.0037201913073658943
Epoch 463, Loss: 0.10114138573408127, Final Batch Loss: 0.03122819773852825
Epoch 464, Loss: 0.10710581112653017, Final Batch Loss: 0.05310172960162163
Epoch 465, Loss: 0.21594158373773098, Final Batch Loss: 0.15404799580574036
Epoch 466, Loss: 0.062192318961024284, Final Batch Loss: 0.008331124670803547
Epoch 467, Loss: 0.05503017781302333, Final Batch Loss: 0.0013355589471757412
Epoch 468, Loss: 0.06590975914150476, Final Batch Loss: 0.0058830492198467255
Epoch 469, Loss: 0.0926882540807128, Final Batch Loss: 0.05162430554628372

...

Epoch 4557, Loss: 0.0001600235176510978, Final Batch Loss: 1.01013790754223e-06
Epoch 4558, Loss: 0.00010007445780502167, Final Batch Loss: 2.139692696800921e-05
Epoch 4559, Loss: 7.20592504421802e-05, Final Batch Loss: 1.6061637779785087e-06
Epoch 4560, Loss: 0.0001094114607838037, Final Batch Loss: 1.1293499824205355e-07
Epoch 4561, Loss: 0.0011287590489246213, Final Batch Loss: 1.9575265923776897e-06
Epoch 4562, Loss: 0.00012375038195955312, Final Batch Loss: 3.199819786914304e-07
Epoch 4563, Loss: 0.00013734410822507925, Final Batch Loss: 0.00011470492609078065
Epoch 4564, Loss: 0.0007227647729450837, Final Batch Loss: 1.6687634342815727e-05
Epoch 4565, Loss: 0.00017797217151382938, Final Batch Loss: 0.00010676262900233269
Epoch 4566, Loss: 0.00018631032127913727, Final Batch Loss: 3.137085613502677e-08
Epoch 4567, Loss: 0.0001356974868258476, Final Batch Loss: 3.137085968774045e-08
Epoch 4568, Loss: 0.00013822845289723773, Final Batch Loss: 1.6297801266773604e-05
Epoch 4569, Loss:

Epoch 4669, Loss: 0.0003603703561338989, Final Batch Loss: 1.4618752857131767e-06
Epoch 4670, Loss: 0.0001321341810580634, Final Batch Loss: 3.7740497646154836e-05
Epoch 4671, Loss: 0.00017557176761329174, Final Batch Loss: 4.0406168409390375e-05
Epoch 4672, Loss: 0.0011568412537599215, Final Batch Loss: 4.178005110588856e-05
Epoch 4673, Loss: 0.00018803277635015547, Final Batch Loss: 1.7121170458267443e-05
Epoch 4674, Loss: 0.0004796205994352931, Final Batch Loss: 2.3034948753775097e-05
Epoch 4675, Loss: 0.00011926913693116603, Final Batch Loss: 1.5685408527588152e-07
Epoch 4676, Loss: 0.0003345503610034939, Final Batch Loss: 0.00012340072134975344
Epoch 4677, Loss: 0.00030470137198790326, Final Batch Loss: 5.445717306429287e-06
Epoch 4678, Loss: 0.00037396361767605413, Final Batch Loss: 1.9371329472051002e-05
Epoch 4679, Loss: 0.00018183648718661516, Final Batch Loss: 1.6940249736308033e-07
Epoch 4680, Loss: 0.0009637835755711421, Final Batch Loss: 0.0006453182431869209
Epoch 4681, L

Epoch 4776, Loss: 0.0019518083609000314, Final Batch Loss: 0.0017164679011330009
Epoch 4777, Loss: 0.0005087996978545561, Final Batch Loss: 0.00017419661162421107
Epoch 4778, Loss: 7.404926691378932e-05, Final Batch Loss: 2.333943484700285e-06
Epoch 4779, Loss: 0.00027117091303807683, Final Batch Loss: 4.658675970858894e-05
Epoch 4780, Loss: 0.02177010467130458, Final Batch Loss: 3.377571192686446e-05
Epoch 4781, Loss: 0.00012701951527560595, Final Batch Loss: 3.6490760976448655e-05
Epoch 4782, Loss: 0.00011492032024307264, Final Batch Loss: 1.2673739320234745e-06
Epoch 4783, Loss: 0.00027620540095085744, Final Batch Loss: 7.974223990458995e-06
Epoch 4784, Loss: 0.0019426078961259918, Final Batch Loss: 2.0516181393759325e-06
Epoch 4785, Loss: 0.0008702447991026929, Final Batch Loss: 9.66214088293782e-07
Epoch 4786, Loss: 0.00014727972883576967, Final Batch Loss: 1.254833392749788e-07
Epoch 4787, Loss: 0.00041816224893409526, Final Batch Loss: 6.618908628297504e-06
Epoch 4788, Loss: 0.0

Epoch 4880, Loss: 0.0009602560930943582, Final Batch Loss: 1.573955887579359e-05
Epoch 4881, Loss: 5.809465706363426e-05, Final Batch Loss: 1.5685414211930038e-07
Epoch 4882, Loss: 0.0008018744716764559, Final Batch Loss: 1.1732641951311962e-06
Epoch 4883, Loss: 0.0002154611393052619, Final Batch Loss: 9.05301931197755e-06
Epoch 4884, Loss: 0.0013056791085546138, Final Batch Loss: 3.0252953365561552e-05
Epoch 4885, Loss: 0.011817212296591606, Final Batch Loss: 0.0015862858854234219
Epoch 4886, Loss: 0.00020472773849178338, Final Batch Loss: 1.4102462046139408e-05
Epoch 4887, Loss: 0.00022920478386367904, Final Batch Loss: 9.682891686679795e-05
Epoch 4888, Loss: 7.614210601047944e-05, Final Batch Loss: 4.3919200720665685e-08
Epoch 4889, Loss: 0.009168521683477593, Final Batch Loss: 3.5385169212531764e-06
Epoch 4890, Loss: 0.0079981161857412, Final Batch Loss: 6.2741727369086675e-09
Epoch 4891, Loss: 0.0010594397408567602, Final Batch Loss: 1.5432751752086915e-05
Epoch 4892, Loss: 0.0506

Epoch 4989, Loss: 0.00021546075115708163, Final Batch Loss: 6.274169805919882e-08
Epoch 4990, Loss: 0.000176805133378366, Final Batch Loss: 4.897849794360809e-05
Epoch 4991, Loss: 8.955323698955908e-05, Final Batch Loss: 2.8861120426881826e-07
Epoch 4992, Loss: 0.0006410646419681143, Final Batch Loss: 0.00028556439792737365
Epoch 4993, Loss: 0.00018428438814055426, Final Batch Loss: 1.8822518654815212e-08
Epoch 4994, Loss: 0.0017149816085293423, Final Batch Loss: 1.2402579159243032e-05
Epoch 4995, Loss: 0.0015182484967226628, Final Batch Loss: 0.0003077991714235395
Epoch 4996, Loss: 0.0004284328023231865, Final Batch Loss: 1.1607097576415981e-06
Epoch 4997, Loss: 0.0011497905943542719, Final Batch Loss: 0.0005070460611023009
Epoch 4998, Loss: 0.000359339026545058, Final Batch Loss: 5.996534673613496e-05
Epoch 4999, Loss: 0.0019265373814505438, Final Batch Loss: 3.2123173241416225e-06
Epoch 5000, Loss: 0.0025719488330651075, Final Batch Loss: 0.0005853799520991743


In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[53  0  0]
 [ 1 35  0]
 [ 0  0 41]]
              precision    recall  f1-score   support

           0    0.98148   1.00000   0.99065        53
           1    1.00000   0.97222   0.98592        36
           2    1.00000   1.00000   1.00000        41

    accuracy                        0.99231       130
   macro avg    0.99383   0.99074   0.99219       130
weighted avg    0.99245   0.99231   0.99229       130



In [11]:
gen = Generator(z_dim = 107)
load_model(gen, "3 Label 4 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=107, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=37, bias=True)
    (4): Tanh()
  )
)

In [13]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 4)

to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()

In [14]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(act_vectors[0], preds.cpu()))
print(metrics.classification_report(act_vectors[0], preds.cpu(), digits = 5))

[[46  0  0]
 [ 0 41  0]
 [ 0  0 43]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        46
           1    1.00000   1.00000   1.00000        41
           2    1.00000   1.00000   1.00000        43

    accuracy                        1.00000       130
   macro avg    1.00000   1.00000   1.00000       130
weighted avg    1.00000   1.00000   1.00000       130



# User Classifier

In [15]:
activities = [1, 3, 4]
users = [1, 3, 5, 7]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [16]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    elif y[k] == 5:
        y[k] = 2
    else:
        y[k] = 3

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Subject_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [18]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.208044171333313, Final Batch Loss: 1.402596354484558
Epoch 2, Loss: 4.160281300544739, Final Batch Loss: 1.3558616638183594
Epoch 3, Loss: 4.2102391719818115, Final Batch Loss: 1.4122371673583984
Epoch 4, Loss: 4.213953375816345, Final Batch Loss: 1.4172205924987793
Epoch 5, Loss: 4.233203411102295, Final Batch Loss: 1.4402642250061035
Epoch 6, Loss: 4.190505862236023, Final Batch Loss: 1.3955961465835571
Epoch 7, Loss: 4.167497754096985, Final Batch Loss: 1.3736603260040283
Epoch 8, Loss: 4.163690686225891, Final Batch Loss: 1.370682716369629
Epoch 9, Loss: 4.129807710647583, Final Batch Loss: 1.3387665748596191
Epoch 10, Loss: 4.194479823112488, Final Batch Loss: 1.408141851425171
Epoch 11, Loss: 4.176658630371094, Final Batch Loss: 1.3896214962005615
Epoch 12, Loss: 4.144842863082886, Final Batch Loss: 1.3559861183166504
Epoch 13, Loss: 4.180180668830872, Final Batch Loss: 1.3935099840164185
Epoch 14, Loss: 4.1601643562316895, Final Batch Loss: 1.3777449131011963
Ep

Epoch 123, Loss: 2.1412729620933533, Final Batch Loss: 0.7736306190490723
Epoch 124, Loss: 1.9125550389289856, Final Batch Loss: 0.5828835368156433
Epoch 125, Loss: 1.9060199856758118, Final Batch Loss: 0.5330233573913574
Epoch 126, Loss: 1.9019542932510376, Final Batch Loss: 0.5371094346046448
Epoch 127, Loss: 2.088755249977112, Final Batch Loss: 0.7455096244812012
Epoch 128, Loss: 1.9948646426200867, Final Batch Loss: 0.6502655148506165
Epoch 129, Loss: 1.8843448162078857, Final Batch Loss: 0.5824071168899536
Epoch 130, Loss: 1.9028897881507874, Final Batch Loss: 0.6064978241920471
Epoch 131, Loss: 2.111026108264923, Final Batch Loss: 0.798348605632782
Epoch 132, Loss: 1.9236676692962646, Final Batch Loss: 0.6375451683998108
Epoch 133, Loss: 2.010519802570343, Final Batch Loss: 0.710108757019043
Epoch 134, Loss: 1.8861061334609985, Final Batch Loss: 0.6005576252937317
Epoch 135, Loss: 1.8728023171424866, Final Batch Loss: 0.5718944668769836
Epoch 136, Loss: 1.7501850426197052, Final 

Epoch 235, Loss: 1.1489294618368149, Final Batch Loss: 0.20311687886714935
Epoch 236, Loss: 1.1390935778617859, Final Batch Loss: 0.28147727251052856
Epoch 237, Loss: 1.5429586470127106, Final Batch Loss: 0.6349425315856934
Epoch 238, Loss: 1.26106595993042, Final Batch Loss: 0.4212353825569153
Epoch 239, Loss: 1.0384783297777176, Final Batch Loss: 0.16659601032733917
Epoch 240, Loss: 1.070555567741394, Final Batch Loss: 0.20458483695983887
Epoch 241, Loss: 1.2731884717941284, Final Batch Loss: 0.42690518498420715
Epoch 242, Loss: 1.3072589635849, Final Batch Loss: 0.5241759419441223
Epoch 243, Loss: 1.282065510749817, Final Batch Loss: 0.43902191519737244
Epoch 244, Loss: 1.2781204283237457, Final Batch Loss: 0.5507075786590576
Epoch 245, Loss: 1.1333045959472656, Final Batch Loss: 0.252521276473999
Epoch 246, Loss: 1.6321991682052612, Final Batch Loss: 0.7898273468017578
Epoch 247, Loss: 1.1582668125629425, Final Batch Loss: 0.42522963881492615
Epoch 248, Loss: 1.3491166532039642, Fi

Epoch 359, Loss: 1.0307234823703766, Final Batch Loss: 0.28586071729660034
Epoch 360, Loss: 0.9307059645652771, Final Batch Loss: 0.24113401770591736
Epoch 361, Loss: 1.0044749975204468, Final Batch Loss: 0.36378034949302673
Epoch 362, Loss: 0.9070173799991608, Final Batch Loss: 0.29491376876831055
Epoch 363, Loss: 0.9696199893951416, Final Batch Loss: 0.3686927556991577
Epoch 364, Loss: 0.9446964263916016, Final Batch Loss: 0.30767402052879333
Epoch 365, Loss: 1.0700596868991852, Final Batch Loss: 0.42744407057762146
Epoch 366, Loss: 1.0144591927528381, Final Batch Loss: 0.3881155252456665
Epoch 367, Loss: 0.940050333738327, Final Batch Loss: 0.28455212712287903
Epoch 368, Loss: 0.833105206489563, Final Batch Loss: 0.14013025164604187
Epoch 369, Loss: 1.1234477162361145, Final Batch Loss: 0.5276561975479126
Epoch 370, Loss: 0.8378441333770752, Final Batch Loss: 0.1881960928440094
Epoch 371, Loss: 1.1107375621795654, Final Batch Loss: 0.4156854450702667
Epoch 372, Loss: 0.9579071700572

Epoch 484, Loss: 0.9442857205867767, Final Batch Loss: 0.28569501638412476
Epoch 485, Loss: 0.8273690193891525, Final Batch Loss: 0.20687805116176605
Epoch 486, Loss: 1.1686211377382278, Final Batch Loss: 0.5942797660827637
Epoch 487, Loss: 1.1133001446723938, Final Batch Loss: 0.5731255412101746
Epoch 488, Loss: 0.9863724708557129, Final Batch Loss: 0.4183967411518097
Epoch 489, Loss: 0.6530584096908569, Final Batch Loss: 0.08807897567749023
Epoch 490, Loss: 0.8643142580986023, Final Batch Loss: 0.32998886704444885
Epoch 491, Loss: 0.8614096939563751, Final Batch Loss: 0.29696711897850037
Epoch 492, Loss: 0.8462692648172379, Final Batch Loss: 0.29615285992622375
Epoch 493, Loss: 0.7559652477502823, Final Batch Loss: 0.16521187126636505
Epoch 494, Loss: 0.8531830459833145, Final Batch Loss: 0.2753623127937317
Epoch 495, Loss: 0.9107130467891693, Final Batch Loss: 0.31840914487838745
Epoch 496, Loss: 0.9552312791347504, Final Batch Loss: 0.4325924515724182
Epoch 497, Loss: 0.95733192563

...

Epoch 4511, Loss: 0.24037155508995056, Final Batch Loss: 0.08937181532382965
Epoch 4512, Loss: 0.1520273475907743, Final Batch Loss: 0.002282956149429083
Epoch 4513, Loss: 0.2115657739341259, Final Batch Loss: 0.03688884153962135
Epoch 4514, Loss: 0.4479920230805874, Final Batch Loss: 0.2952471673488617
Epoch 4515, Loss: 0.21572491899132729, Final Batch Loss: 0.04017079249024391
Epoch 4516, Loss: 0.29244598746299744, Final Batch Loss: 0.13547734916210175
Epoch 4517, Loss: 0.2039421834051609, Final Batch Loss: 0.06500119715929031
Epoch 4518, Loss: 0.2531706467270851, Final Batch Loss: 0.08472727984189987
Epoch 4519, Loss: 0.18792669917456806, Final Batch Loss: 0.0019534423481673002
Epoch 4520, Loss: 0.19741864129900932, Final Batch Loss: 0.05658482760190964
Epoch 4521, Loss: 0.23829424753785133, Final Batch Loss: 0.031469862908124924
Epoch 4522, Loss: 0.2594878822565079, Final Batch Loss: 0.08018467575311661
Epoch 4523, Loss: 0.20637282356619835, Final Batch Loss: 0.05936063081026077
Ep

Epoch 4620, Loss: 0.36197205632925034, Final Batch Loss: 0.24911701679229736
Epoch 4621, Loss: 0.1576077975332737, Final Batch Loss: 0.008135493844747543
Epoch 4622, Loss: 0.24119500070810318, Final Batch Loss: 0.15481379628181458
Epoch 4623, Loss: 0.24139203503727913, Final Batch Loss: 0.11223342269659042
Epoch 4624, Loss: 0.5782293975353241, Final Batch Loss: 0.41750651597976685
Epoch 4625, Loss: 0.1519104866310954, Final Batch Loss: 0.015086228959262371
Epoch 4626, Loss: 0.5795630291104317, Final Batch Loss: 0.3446328639984131
Epoch 4627, Loss: 0.20627234131097794, Final Batch Loss: 0.025598518550395966
Epoch 4628, Loss: 0.16356072947382927, Final Batch Loss: 0.006253283470869064
Epoch 4629, Loss: 0.1800444582477212, Final Batch Loss: 0.006047391332685947
Epoch 4630, Loss: 0.21989133208990097, Final Batch Loss: 0.07737696915864944
Epoch 4631, Loss: 0.2131650261580944, Final Batch Loss: 0.03965600207448006
Epoch 4632, Loss: 0.09539476223289967, Final Batch Loss: 0.01830897107720375
E

Epoch 4738, Loss: 0.09806342422962189, Final Batch Loss: 0.008512957021594048
Epoch 4739, Loss: 0.3645642362535, Final Batch Loss: 0.2588719427585602
Epoch 4740, Loss: 0.10964975878596306, Final Batch Loss: 0.014916520565748215
Epoch 4741, Loss: 0.10112450085580349, Final Batch Loss: 0.012999607250094414
Epoch 4742, Loss: 0.342053584754467, Final Batch Loss: 0.27294009923934937
Epoch 4743, Loss: 0.3186875879764557, Final Batch Loss: 0.1813517063856125
Epoch 4744, Loss: 0.13520499505102634, Final Batch Loss: 0.01556633971631527
Epoch 4745, Loss: 0.07826047856360674, Final Batch Loss: 0.013366187922656536
Epoch 4746, Loss: 0.11267565563321114, Final Batch Loss: 0.02159157395362854
Epoch 4747, Loss: 0.1253312360495329, Final Batch Loss: 0.02185080759227276
Epoch 4748, Loss: 0.12817781418561935, Final Batch Loss: 0.008887097239494324
Epoch 4749, Loss: 0.25750797614455223, Final Batch Loss: 0.20123685896396637
Epoch 4750, Loss: 0.1535705290734768, Final Batch Loss: 0.052171044051647186
Epoc

Epoch 4861, Loss: 0.19886874593794346, Final Batch Loss: 0.01898898370563984
Epoch 4862, Loss: 0.1373995179310441, Final Batch Loss: 0.008958426304161549
Epoch 4863, Loss: 0.39793575555086136, Final Batch Loss: 0.2688280940055847
Epoch 4864, Loss: 0.1483327429741621, Final Batch Loss: 0.02845117636024952
Epoch 4865, Loss: 0.20396841689944267, Final Batch Loss: 0.053159449249506
Epoch 4866, Loss: 0.17140673100948334, Final Batch Loss: 0.06954295933246613
Epoch 4867, Loss: 0.17469352670013905, Final Batch Loss: 0.03064718283712864
Epoch 4868, Loss: 0.14128515776246786, Final Batch Loss: 0.011528155766427517
Epoch 4869, Loss: 0.2205457855015993, Final Batch Loss: 0.007787054404616356
Epoch 4870, Loss: 0.11820053122937679, Final Batch Loss: 0.031036389991641045
Epoch 4871, Loss: 0.19148256815969944, Final Batch Loss: 0.01610925979912281
Epoch 4872, Loss: 0.1355189885944128, Final Batch Loss: 0.019263116642832756
Epoch 4873, Loss: 0.17470603808760643, Final Batch Loss: 0.08667387813329697
E

Epoch 4969, Loss: 0.3180893659591675, Final Batch Loss: 0.02424507588148117
Epoch 4970, Loss: 0.2514737723977305, Final Batch Loss: 0.00035335408756509423
Epoch 4971, Loss: 0.14888514950871468, Final Batch Loss: 0.03466518595814705
Epoch 4972, Loss: 0.18993560131639242, Final Batch Loss: 0.01429308857768774
Epoch 4973, Loss: 0.31204796209931374, Final Batch Loss: 0.18517427146434784
Epoch 4974, Loss: 0.13414493575692177, Final Batch Loss: 0.012211695313453674
Epoch 4975, Loss: 0.20877769961953163, Final Batch Loss: 0.06065962091088295
Epoch 4976, Loss: 0.2125321663916111, Final Batch Loss: 0.05060240998864174
Epoch 4977, Loss: 0.15165827609598637, Final Batch Loss: 0.018443042412400246
Epoch 4978, Loss: 0.18764149770140648, Final Batch Loss: 0.04203500226140022
Epoch 4979, Loss: 0.6482780501246452, Final Batch Loss: 0.4987388849258423
Epoch 4980, Loss: 0.1639898531138897, Final Batch Loss: 0.02782561257481575
Epoch 4981, Loss: 0.25602205097675323, Final Batch Loss: 0.1259150356054306
E

In [19]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[42  0  0  0]
 [ 1 28  3  1]
 [ 0  1 24  0]
 [ 0  0  0 30]]
              precision    recall  f1-score   support

           0    0.97674   1.00000   0.98824        42
           1    0.96552   0.84848   0.90323        33
           2    0.88889   0.96000   0.92308        25
           3    0.96774   1.00000   0.98361        30

    accuracy                        0.95385       130
   macro avg    0.94972   0.95212   0.94954       130
weighted avg    0.95492   0.95385   0.95306       130



In [20]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(usr_vectors[0], preds.cpu()))
print(metrics.classification_report(usr_vectors[0], preds.cpu(), digits = 5))

[[39  0  0  0]
 [ 0 34  0  0]
 [ 0  0 25  0]
 [ 0  0  0 32]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        39
           1    1.00000   1.00000   1.00000        34
           2    1.00000   1.00000   1.00000        25
           3    1.00000   1.00000   1.00000        32

    accuracy                        1.00000       130
   macro avg    1.00000   1.00000   1.00000       130
weighted avg    1.00000   1.00000   1.00000       130

